In [2]:
from typing import Dict, Optional
from datasets import Dataset, load_dataset, concatenate_datasets
import os
import glob
import json
import copy
import threading
import time
import random
from typing import Dict, Optional
import requests
import numpy as np
import re

import fasttext
lang_detect = fasttext.load_model('../fastchat/modules/fasttext/lid.176.bin')

from fastchat.modules.answer_refiner import generate_refiner

import chromadb
from chromadb.config import Settings
import random
from fastchat.modules.embedder_adapter import Embedder, get_embedder
from fastchat.conversation import (
    SeparatorStyle,
)
from fastchat.model.model_adapter import get_conversation_template
import copy
from fastchat.train.data_modules.sft_dataset import load_sft_dataset, combine_dataset
from fastchat.train.data_modules.dpo_dataset import load_dpo_dataset
from fastchat.train.data_modules.dedup import (
    dedup_by_similarity,
    dedup_non_pair,
    dedup_repetition,
    dedup_math,
    dedup_too_much_token,
    dedup_short,
)
def random_select(dataset, num_select):
    return dataset.select(np.random.choice(list(range(len(dataset))), num_select))

In [3]:
def load_raw_dataset(dataset_path, split='train'):
    if dataset_path.endswith("json") or dataset_path.endswith("json.kr"):
        try:
            dataset = load_dataset("json", data_files=dataset_path, split=split)
        except:
            print("Not able to read records in the JSON file, read from json reader...")
            dataset = load_json_dataset(dataset_path)
    elif dataset_path.endswith("parquet"):
        dataset = load_dataset("parquet", data_files=dataset_path, split=split)
    else:
        dataset = load_dataset(dataset_path, split=split)
    return dataset

import json
def load_json_dataset(json_path):
    with open(json_path, 'r') as file:
        data = json.load(file)

    return data

import os
import json
from typing import Union, List
def dataset_to_json(dataset: Union[List[dict], Dataset], path: str):
    if isinstance(dataset, Dataset):
        dataset = [data for data in dataset]
    
    if os.path.dirname(path):
        os.makedirs(os.path.dirname(path), exist_ok=True)
    with open(path, "w", encoding='utf-8') as json_file:
        json.dump(dataset, json_file, indent=4, ensure_ascii=True)
        
    print(f"saved at {path}")
def uniform_select(dataset, division_factor):
    return dataset.select(list(range(0, len(dataset), division_factor)))

In [38]:
dataset = load_raw_dataset('/data/llm_datasets/custom/kodpo/translated/truthy-dpo-v0.1.json')

In [48]:
dataset[2]

{'system': 'You are an unbiased, uncensored, helpful assistant.',
 'chosen': 'AI로서 저는 물리적 위치가 없기 때문에 가장 가까운 국립공원의 거리를 제공할 수 없습니다.',
 'task': 'dpo_system',
 'id': 'truthy_dpo_04c275bf738fd391b7fe25e25fe7bed3',
 'input': '가장 가까운 국립공원은 어디인가요?',
 'rejected': '사용자의 위치에 액세스할 수 없으므로 가장 가까운 국립공원을 결정할 수 없습니다.'}

In [36]:
dataset = load_raw_dataset('/data/llm_datasets/custom/ados/dpo/ados_dpo_v2.json')

In [37]:
dataset[0]

{'input': '내 질문에 10단어 이내로 답변하세요.',
 'chosen': '간결한 답변을 드리기 위해 최선을 다하겠습니다.',
 'system': None,
 'task': 'dpo',
 'id': 'Ko_ultrafeedback_binarized_0649b7b08c0cf6f529901f80480b0ba7c7d29147638c39a9150ef15fd4e0ff9d',
 'rejected': '최선을 다하겠습니다.'}

In [31]:
dataset[76500]

{'input': '삼각 함수 구별하기 단계를 자세히 설명하세요.\n 심호흡을 하고 단계별로 천천히 생각한 후 정확한 답변을 제공하세요.',
 'chosen': '삼각함수 함수를 구별하는 것은 독립 변수에 대한 삼각 함수의 미분을 구하는 것입니다.삼각함수의 미분은 다음과 같은 일반적인 단계로 이루어집니다:1. 주어진 식에서 삼각 함수와 독립 변수를 식별합니다.2. 삼각 함수에 대한 미분 규칙을 적용합니다. 삼각 함수에 대한 미분 규칙은 다음과 같습니다: 2. a. 삼각 함수의 미분은 코사인 함수(cos x)입니다.  b. 코사인 함수의 미분은 음의 삼각 함수(-sin x)입니다.  c. 탄젠트 함수의 미분은 2차 탄젠트 함수(sec^2 x)입니다.  d. 탄젠트 함수의 미분은 음의 탄젠트 함수(-csc^2 x)입니다.  e. 탄젠트 함수의 미분은 탄젠트 함수(tan x)의 탄젠트 함수 곱하기 탄젠트 함수(cot x).  f. 탄젠트 함수의 미분은 음의 탄젠트 함수(-csc x) 곱하기 탄젠트 함수(cot x)입니다.3. 미분을 적용한 후 식을 단순화합니다.4. 삼각 함수가 다른 함수의 사인 또는 코사인처럼 다른 함수가 중첩되어 있는 경우, 중첩 함수를 미분하기 위해 연쇄 규칙을 적용해야 할 수도 있습니다.연쇄 규칙을 사용할 때는 신중하게 사용하고 필요에 따라만 연쇄 규칙을 적용해야 합니다.항상 가능한 단순화 단계를 확인하세요.삼각 함수가 다른 함수와 관련된 더 큰 식의 일부인 경우 삼각 함수를 구별하는 과정이 더 복잡해질 수 있습니다. 이러한 경우 추가 규칙과 기법이 필요할 수 있습니다.삼각 함수가 생각나는 것이 있으면 알려주시면 자세한 단계별 해결책을 제공해 드리겠습니다.',
 'system': None,
 'task': 'dpo',
 'id': 'Ko_distilabel-math-preference-dpo_2207',
 'rejected': '삼각함수를 구별하려면 다음 단계를 따르세요:\n\n1. 삼각함수와 관련된 삼각함수를 식별합니다. 가장 일반적인 삼각

In [ ]:
from collections import defaultdict

"""
get and save language distributions
"""
def get_lang_distribution(dataset_list):
    lang_distribution = {}
    global_lang_dict = {'train': defaultdict(int), 'test': defaultdict(int)}
    for dataset_path in dataset_list:
        # dataset = load_sft_dataset(dataset_path, split=None)
        dataset = load_dpo_dataset(dataset_path, split=None)

        lang_splits = {}
        for split in list(dataset.keys()):
            print(f"{dataset_path}:{split}")
            _dataset = dataset[split]

            lang_dict = defaultdict(int)
            for data in _dataset:
                # conversations = data['conversations']
                conversations = data['chosen_response']
                langs = {}
                len_conv = 0
#                 for conv in conversations:
#                     # _from = conv['from']
#                     # _value = conv['value']
#                     _from = conv['role']
#                     _value = conv['content']

#                     len_conv += len(_value)
#                     lang, conf = lang_detect.predict(_value.replace('\n', ' '))
#                     lang = lang[0]
#                     if lang not in langs:
#                         langs[lang] = 1
#                     else:
#                         langs[lang] += 1
                _value = conversations

                len_conv += len(_value)
                lang, conf = lang_detect.predict(_value.replace('\n', ' '))
                lang = lang[0]
                if lang not in langs:
                    langs[lang] = 1
                else:
                    langs[lang] += 1

#                 if '__label__en' in langs:
#                     langs['__label__en'] -= 1

                if len(langs) == 0:
                    dominent_lang = "empty"
                else:
                    dominent_lang = max(langs)
                
                # if dominent_lang not in lang_dict:
                #     lang_dict[dominent_lang] = 1
                # else:
                global_lang_dict[split][dominent_lang] += 1
                lang_dict[dominent_lang] += 1
            lang_splits[split] = lang_dict
        lang_distribution[dataset_path] = lang_splits
    
    lang_distribution['total'] = global_lang_dict
    
    stat_dict = {'train': {}, 'test': {}}
    for split in ['train', 'test']:
        total_cnt = sum([value for value in global_lang_dict[split].values()])
        for key, value in global_lang_dict[split].items():
            stat_dict[split][key] = f"{value / total_cnt:.2%}"
    
    lang_distribution['stat'] = stat_dict
    
    return lang_distribution

lang_distribution = get_lang_distribution([dpo_list[2]])
lang_distribution

In [6]:
with open("/workspaces/data/llm_datasets/custom/lang_distribution_SFT_v4.json", "w") as json_file:
    json.dump(lang_distribution, json_file, indent=4)

In [ ]:
dataset = load_dataset("json", data_files="/workspaces/data/llm_datasets/custom/deduped/translated_sharegpt_V3_format_ko.json")
dataset

In [ ]:
dataset_koen = dataset['train'].select(range(10000, 15000))
dataset_enko = dataset['train'].select(range(15000, 20000))

In [ ]:
"""
SFT raw data and change format
"""

In [ ]:
new_dataset = []
for data in dataset_enko:
    conversations_ko = data['conversations']
    _id = data['id']
    
    conversations_en = lang_dict['__label__en'][_id]['conversations']
    
    for _idx in range(len(conversations_ko)):
        value_ko = conversations_ko[_idx]['value']
        value_en = conversations_en[_idx]['value']
        if len(value_ko) > 150: # 너무 짧은 데이터는 품질이 좋지 않아 제거
            new_dataset.append({
                'id': f"sharegpt_V3_format_{_id}_{_idx}",
                'task': 'enkotranslation',
                'conversations': [
                                    {'from': 'human', 'value': value_en},
                                    {'from': 'gpt', 'value': value_ko},
                                 ],
            })

In [ ]:
"""make lang_dict(sft)"""
dataset = load_dataset("json", data_files="/data/llm_datasets/custom/kodpo/untranslated/ultrafeedback_binarized.json")
enko_dataset = []
remained_dataset = []

lang_dict = {}
for data in dataset['train']:
    conversations = data['conversations']
    langs = {}
    len_conv = 0
    for conv in conversations:
        _from = conv['from']
        _value = conv['value']

        len_conv += len(_value)
        lang, conf = lang_detect.predict(_value.replace('\n', ' '))
        lang = lang[0]
        if lang not in langs:
            langs[lang] = 1
        else:
            langs[lang] += 1

    if '__label__en' in langs:
        langs['__label__en'] -= 1

    dominent_lang = max(langs)
    if dominent_lang not in lang_dict:
        lang_dict[dominent_lang] = [data]
    else:
        lang_dict[dominent_lang].append(data)
    # break

In [16]:
"""make lang_dict(dpo)"""
dataset = load_dataset("json", data_files="/data/llm_datasets/custom/kodpo/untranslated/truthy-dpo-v0.1.json")
enko_dataset = []
remained_dataset = []

lang_dict = {}
for data in dataset['train']:
    chosen = data['chosen']
    rejected = data['rejected']
    
    langs = {}

    lang_chosen = lang_detect.predict(chosen.replace('\n', ' '))[0][0]
    if lang_chosen not in langs:
        langs[lang_chosen] = 1
    else:
        langs[lang_chosen] += 1
        
    lang_rejected = lang_detect.predict(rejected.replace('\n', ' '))[0][0]
    if lang_rejected not in langs:
        langs[lang_rejected] = 1
    else:
        langs[lang_rejected] += 1

    if '__label__en' in langs:
        langs['__label__en'] -= 1

    dominent_lang = max(langs)
    if dominent_lang not in lang_dict:
        lang_dict[dominent_lang] = [data]
    else:
        lang_dict[dominent_lang].append(data)
    # break

Extracting data files: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 902.97it/s]
Generating train split: 1016 examples [00:00, 21155.38 examples/s]


In [17]:
for key, value in lang_dict.items():
    print(key, len(value))

__label__en 1016


In [ ]:
import requests
api_server_url = "http://localhost:41002"
def count_total_token(conversations):
    num_token = 0
    for conv in conversations:
        input_json = {
            "model_name": "MingAI-70B-chat-orca_v0.42_2_dpo-GPTQ",
            "prompt": conv['value'],
        }

        ret = requests.post(api_server_url + "/count_token", json=input_json)

        output_json = ret.json()
        num_token += output_json['count']
    return num_token


for _idx in range(10):
    num_total_token = count_total_token(lang_dict['__label__en'][_idx])
    print(num_total_token)

In [ ]:
new_dataset += lang_dict['__label__pt'] + lang_dict['__label__es'] + lang_dict['__label__de'] + lang_dict['__label__zh'] + lang_dict['__label__fr']

In [ ]:
with open("/data/llm_datasets/custom/vicuna_format/lima_vicuna_format_ko.json", "w") as json_file:
    json.dump(new_dataset, json_file)

In [ ]:
new_dataset[2]

In [118]:
id_row = 'Ko_distilabel-math-preference-dpo_1'
filtered_row = dataset.filter(lambda sample: sample['id'] == id_row)
filtered_row[0]

Filter: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2418/2418 [00:00<00:00, 111092.18 examples/s]


{'input': 'What is the result of substituting `x = 5` and `y = -2` into the expression `2x^2 - 3y + 7`?.\n Take a deep breath, think step by step, and give an accurate response',
 'chosen': "To find the result of substituting `x = 5` and `y = -2` into the expression `2x^2 - 3y + 7`, we will substitute these values for the variables `x` and `y` in the given expression and simplify.\n\nSubstituting `x = 5` and `y = -2` into the expression, we get:\n\n2(5)^2 - 3(-2) + 7\n\nSimplifying, we have:\n\n2(25) + 6 + 7\n\nNow, let's perform the operations:\n\n50 + 6 + 7 = 63\n\nTherefore, the result of substituting `x = 5` and `y = -2` into the expression `2x^2 - 3y + 7` is 63.",
 'task': 'dpo',
 'id': 'Ko_distilabel-math-preference-dpo_1',
 'rejected': 'To find the result of substituting `x = 5` and `y = -2` into the expression `2x^2 - 3y + 7`, we need to replace `x` with `5` and `y` with `-2` in the expression and simplify it. \n\nSubstituting `x = 5` and `y = -2`:\n2(5)^2 - 3(-2) + 7\n\nSimpli

In [44]:
id_row = 'Ko_ultrafeedback_binarized_0649b7b08c0cf6f529901f80480b0ba7c7d29147638c39a9150ef15fd4e0ff9d'
filtered_row = temp_dataset.filter(lambda sample: sample['id'] == id_row)
filtered_row[0]

Filter: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 46462/46462 [00:01<00:00, 40620.39 examples/s]


{'rejected': '최선을 다하겠습니다.',
 'id': 'Ko_ultrafeedback_binarized_0649b7b08c0cf6f529901f80480b0ba7c7d29147638c39a9150ef15fd4e0ff9d',
 'chosen': '간결한 답변을 드리기 위해 최선을 다하겠습니다.',
 'input': '내 질문에 10단어 이내로 답변하세요.',
 'task': 'dpo'}

In [ ]:
%%time
"""dpo translate"""
api_server_url = "http://localhost:21122"
def send_translate_request(new_dataset):
    global idx
    # for _ in range(2):
    while(1):
        if idx >= len_dataset:
            break
        lock.acquire()
        pidx = idx
        data = lang_dict['__label__en'][pidx]
        idx += 1
        lock.release()
        
        print(f"{idx}/{len_dataset}", '\t\t\t\t\t\t', end='\r')#
        
        system = data['system']#option
        _input = data['input']
        chosen = data['chosen']
        rejected = data['rejected']
        
        new_conv = {'system': '', 'input': '', 'chosen': '', 'rejected': ''}
        for vidx, value in enumerate([_input, chosen, rejected, system]):
            if vidx == 3:
                fidx = value.find('\n')
                if fidx != -1:
                    prefix = value[:fidx + 1]
                    value = value[fidx + 1:]
                else:
                    new_conv['system'] = value
                    continue
            
            # response
            results = ""
            text_blocks = []
            code_blocks = []
            for bidx, block in enumerate(value.split("```")):
                if bidx % 2 == 0:
                    text_blocks.append(block)
                else:
                    code_blocks.append(block)

            for tidx, text_block in enumerate(text_blocks):
                prompt = f"### 영어:\n{text_block}\n### 한국어:\n"
                input_json = {
                    "model_name": "Gugugo-koen-7B-V1.1",
                    "prompt": prompt,
                    "temperature": 0.7,
                    "top_p": 0.8,
                    "max_new_tokens": 4096,
                    "stop": ["</끝>", "###"],
                }

                ret = requests.post(
                    api_server_url + "/worker_generate_stream",
                    json=input_json,
                    stream=True,
                )

                for chunk in ret.iter_lines(decode_unicode=False, delimiter=b"\0"):
                    if chunk:
                        result_data = json.loads(chunk.decode())

                result = result_data['text'][len(prompt):].rstrip('\n')
                results += result
                if len(code_blocks) > tidx:
                    results += "```" + code_blocks[tidx] + "```"
                    
            if vidx == 0:
                new_conv['input'] = results
            elif vidx == 1:
                new_conv['chosen'] = results
            elif vidx == 2:
                new_conv['rejected'] = results
            elif vidx == 3:
                new_conv['system'] = prefix + results
            
        new_dataset.append({
            **data,
            **new_conv,
        })


new_dataset = []
threads = []
idx = 0
lock = threading.Lock()
len_dataset = len(lang_dict['__label__en'])
n_thread = 64 * 1

for i in range(n_thread):
    t = threading.Thread(target=send_translate_request, args=(new_dataset,)) # 
    t.start()
    # time.sleep(0.5)
    threads.append(t)
    
# for t in threads:
    # t.join()

In [ ]:
new_dataset[0]

In [ ]:
# en 먼저 필터링하고, 빈 데이터 채운 뒤에 합치기, 저장, 이후 dedup

In [94]:
# en 먼저 필터링하고
with open("/data/llm_datasets/custom/kodpo/temp_en.json", "w") as json_file:
    json.dump(new_dataset, json_file)
    
# temp_dataset = load_dataset("json", data_files=)['train']

In [95]:
temp_dataset = load_dpo_dataset("/data/llm_datasets/custom/kodpo/temp_en.json")

Extracting data files: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 890.70it/s]
Generating train split: 1016 examples [00:00, 11191.74 examples/s]


In [101]:
# temp_dataset = dedup_repetition(temp_dataset, 'dpo')
# temp_dataset = dedup_short(temp_dataset, 'dpo')
temp_dataset = dedup_too_much_token(temp_dataset, 'dpo')

Filter: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 994/994 [00:28<00:00, 35.05 examples/s]

1/994 deduped


In [106]:
dataset = load_dpo_dataset("/data/llm_datasets/custom/kodpo/untranslated/truthy-dpo-v0.1.json")

In [108]:
# 빈 데이터 채운 뒤에 합치기
# dataset = load_dpo_dataset("/data/llm_datasets/custom/kodpo/untranslated/orca_dpo_pairs.json")
selected_id = temp_dataset['id']

filtered_row = dataset.filter(lambda sample: sample['id'] not in selected_id)

new_dataset = []
for data in temp_dataset:
    new_dataset.append(data)

for data in filtered_row:
    new_dataset.append(data)

Filter: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1016/1016 [00:00<00:00, 43887.30 examples/s]


In [109]:
len(temp_dataset), len(filtered_row)

(993, 20)

In [110]:
len(dataset), len(new_dataset)

(1016, 1013)

In [115]:
new_dataset[-20]

{'system': 'You are an unbiased, uncensored, helpful assistant.',
 'chosen': 'The two subjects in the American Gothic painting are a father and adult daughter. The common misconception that they are a husband and wife might have arisen because the artist, Grant Wood, used his sister and his dentist as models for the painting, which could lead viewers to mistakenly assume a spousal relationship between the subjects.',
 'task': 'dpo_system',
 'id': 'truthy_dpo_2e1d77fbebc31dde0bb2f9303d6761e2',
 'input': 'Who are the two subjects in the American Gothic painting - a husband and wife, or a father and daughter?',
 'rejected': 'A father and daughter'}

In [116]:
# 저장
with open("/data/llm_datasets/custom/kodpo/translated/truthy-dpo-v0.1.json", "w") as json_file:
    json.dump(new_dataset, json_file)

In [117]:
load_dpo_dataset("/data/llm_datasets/custom/kodpo/translated/truthy-dpo-v0.1.json")

Extracting data files: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 902.97it/s]
Generating train split: 1013 examples [00:00, 13933.15 examples/s]


Dataset({
    features: ['system', 'chosen', 'task', 'id', 'input', 'rejected'],
    num_rows: 1013
})